In [16]:
# Schritt 1: Bibliotheken importieren
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, Model,  load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, SimpleRNN
from keras.saving import save_model
import time
import os

# Parameter
#symbol = 'TSLA'
pred_days = 100
look_back = 100 #60 -- 

# ====== SETTINGS ======
# 1. Daten abrufen
folderPath_His = "/Users/Shared/ai_work/Trainingdata/ml_data/yh_his/D1/"
folderPath_models = "/Users/Shared/ai_work/Trainingdata/models/lstm_ta/"
folderpath_results = "/Users/Shared/ai_work/Trainingdata/ml_results/lstm_ta/"
# Schritt 5: Daten skalieren und Sequenzen vorbereiten
features = ['open', 'high', 'low', 'close', 'volume']
# ===== END SETTINGS ======

lastBars = 0
n_tail = 400
n_pred_days = 60
n_time_steps = 60 # Neuronal Netzwerk
sequence_length = 60
n_epochs = 100 
n_batch_size = 32 #32

#data = df[['close']].dropna()



In [17]:
def displayChart (future_dates,pred_prices,  data, df_his):
    # Schritt 9: Ergebnisse plotten
    fig = go.Figure()
    df_his = df_his.set_index('date')
    df_his = df_his.tail (lastBars)
    fig.add_trace(go.Scatter(x=data.index[-200:], y=data['close'][-200:], name='Historische Kurse'))
    fig.add_trace(go.Scatter(x=df_his.index, y=df_his['close'],  name='IS HIS'  ))
    fig.add_trace(go.Scatter(x=future_dates, y=pred_prices, name='Prognose (100 Tage)'))
    fig.update_layout(title=   '  Prognose mit LSTM und technischen Indikatoren', xaxis_title='Datum', yaxis_title='Preis (USD)')
    fig.show()


In [18]:
def build_model(X, y, model_name):
    # Schritt 6: LSTM-Modell erstellen
    model = Sequential([
    LSTM(100, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
    Dropout(0.2),
    LSTM(100),
    Dropout(0.2),
    Dense(pred_days)
    ])
    model.compile(optimizer='adam', loss='mse')
    # Schritt 7: Modell trainieren
    model.fit(X, y, epochs=n_epochs, batch_size=n_batch_size, verbose=1)
    
    save_model(model, model_name)
    return model 


In [19]:
def predictBySymbol (symbol):
    
    df = pd.read_csv (folderPath_His + symbol + ".csv")
    df_his = df.tail(lastBars)
    df_his["date"] = pd.to_datetime(df_his["date"]) 
    df = df[['date','open', 'high', 'low', 'close','volume']]
    df['date'] = pd.to_datetime(df['date'])
    df = df.set_index('date')
    data = df.copy()
    #data = df.copy()
    if n_tail > 0: data = data.tail(n_tail)
    if lastBars > 0: data = data[: len(data)-lastBars]
    # Schritt 4: Technische Indikatoren berechnen
    data['SMA_20'] = data['close'].rolling(window=20).mean()
    data['EMA_20'] = data['close'].ewm(span=20, adjust=False).mean()
    change = data['close'].diff()
    gain = change.where(change > 0, 0).rolling(window=14).mean()
    loss = (-change.where(change < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    data['RSI'] = 100 - (100 / (1 + rs))

    ema12 = data['close'].ewm(span=12, adjust=False).mean()
    ema26 = data['close'].ewm(span=26, adjust=False).mean()
    data['MACD'] = ema12 - ema26

    std = data['close'].rolling(window=20).std()
    data['Bollinger_Upper'] = data['SMA_20'] + (std * 2)
    data['Bollinger_lower'] = data['SMA_20'] - (std * 2)

    data.fillna(method='bfill', inplace=True)
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data[features])

    X, y = [], []
    for i in range(look_back, len(scaled_data) - pred_days):
       X.append(scaled_data[i - look_back:i])
       y.append(scaled_data[i:i + pred_days, 3])  # close-Preis
    X, y = np.array(X), np.array(y)

    filePath_model =  folderPath_models +symbol+ ".keras"
    model_name = filePath_model
    model_pred =""
    if os.path.exists(model_name):
       model_pred = load_model(model_name)
    else:
       model_pred = build_model(X, y, model_name)
    # Schritt 8: Vorhersage für die nächsten 100 Handelstage
    last_seq = scaled_data[-look_back:]
    last_seq = np.expand_dims(last_seq, axis=0)

    pred_scaled = model_pred.predict(last_seq)[0]

    scaler_close = MinMaxScaler()
    scaler_close.fit(data[['close']])
    pred_prices = scaler_close.inverse_transform(pred_scaled.reshape(-1, 1)).flatten()

    future_dates = pd.date_range(start=data.index[-1], periods=pred_days+1, freq='B')[1:]
    #display Chart 
    displayChart (future_dates, pred_prices, data, df_his)
    



In [20]:
predictBySymbol ('#PLTR')

/tmp/ipykernel_3476714/864163038.py:30: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
